In [1]:
import gpt_2_simple as gpt2
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np


/Users/jakehenderson/Documents/code/projects/machine-learning-practice/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load GPT-2 model (download if not already downloaded)
gpt2.download_gpt2()

# Load pre-trained SentenceTransformer model
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Fetching checkpoint: 1.05Mit [00:00, 2.36Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 3.05Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 9.20Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:30, 16.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 5.70Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.69Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 4.23Mit/s]                                                      
/Users/jakehenderson/Documents/code/projects/machine-learning-practice/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are

In [3]:
# Sample page of notes (replace this with your actual text)
notes = """
#bubble 
When I write, or use the computer, I want to see some stuff that I have written before. 
I want there to be a recommendation alorithm that points out to me stuff that I have jotted down. 
Quotes, ideas, plans, technical notes. I want to be reminded of when and what I was thinking in the past. 

How am I using reminders?
today, this week, longterm, then specific projects

how am I using notes:
thats really all kinds of things
I have used notion
I have used obsidian

seems like what I want to be able to do is give a command
about what I am writing and be writing in that vein the entire time.

if I say note: I am writing a multiline note.
if I say bubble: I am writing a minimal quick idea.
if I say quote I am writing a quote.

honestly we need a deepwork station that works for us
I don’t want to have to look up or risk breaking focus 

we have to be able to setup commands to do certain other tasks
we have to build an ontology of relatedness

want to be able to say what note or topic

If I want to add to the visions I have I want to be able to say
visions: whatever I am thingking of at the time.

Ocean should have tides
tides are subdivisions
qoutes would be a tide
lets make it just that simple

if I go 
quote:: whatever - Jean-Paul Sartre
hopefully in the UI we can color how each Tide looks 
tides can have a subtide, the tide should really be the unique way that we think of “what we are talking about”
 IF the user has to do too much work to make like things like, its really not helpful. Its obsidian and Notion all over again with the working

It has too feel like this stuff all flies from the users fingertips

basically a tide is a topic, and the bubble is the content
there should be special support for certain types of topic
a quote for example should present as a quote and a subject

writing quote::: should then show the rest that way as you go
“italic whaever the person said” - who they were

bubbles are rust code
what are topics, where do the topics go

its a graph where the edges have names isn’t it
But what is the program
I think there should be 
Ocean:
	topics: Vec<String>
	bubbles: <Vec> Bubble
	what else does the ocean need to do other than convey the state
	the ocean is essentially a cache for which the state of the application is maintained
	it cannot be duplicated and there can be only one instance at a time
	
lets give some thanks here. Here we are using the greatest language ever written to write an app I have
been thinking about building for years now. Life is good. you never know how it will go. Let your anxieties melt
away and 

removing a bubble is an interesting thing. Is there a way we can remove it once and have it be done? What if a bubble has not topic?
is it instantly ocean?

you can make a bubble without a topic
you can make a topic without a bubble
I think its just not a good idea to have 
the topic of a given bubble saved inside the bubble

Removing is really secondary, Its kinda not the point of this stuff
ther persistance part of all of this is a waste of time for a demo anyway

Its really about that UI portion
then do the lookup bullshit
then do the 

Its worth thinking about the bubble to topic relationship think as a graph
Its an adjacentcy list

need a decent text box with a send button, need to know what the fuck we put in there in the first place
will actually be nice to use when we get that shit going for the first time, and I can finally stare into the abyss and think

The app should be a rest API that has one endpoint and goes from there.
I guess it will also need get ocean and such

would even be nice to see this used for taking notes about this very app tbh

can topics have topics?
notes

API
OCEAN (app state)
db -> JSON serde at this point

Talk out the flow
User logs in, sees all their bubbles -> getAllBubbles

API:
get all
write new
parse new
do command

UI should be all about displaying this stuff well
finally data manipulation as you go

There has to be a command interpreter and thus a parser of what the user has written 
Would be really cool if I could get NLP down to let them basically use any basic way to ask for stuff

Have to get voice down. Has to be a quick button click to get a note and topic
"""

In [4]:
import spacy

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

In [5]:
# Tokenize the text into sentences using spaCy
sentences = [sent.text for sent in nlp(notes).sents]

# Generate GPT-2 embeddings for each sentence
def get_sentence_embeddings(sentences):
    return bert_model.encode(sentences)

# Hierarchical clustering
sentence_embeddings = get_sentence_embeddings(sentences)
cluster_model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage="ward").fit(sentence_embeddings)
num_clusters = len(np.unique(cluster_model.labels_))

# Create text files for each cluster with improved subject descriptions
for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i in range(len(cluster_model.labels_)) if cluster_model.labels_[i] == cluster_id]
    
    # Generate a brief subject description using GPT-2
    subject_description = gpt2.generate(sess, prefix=cluster_sentences[0], length=50, temperature=0.7, return_as_list=True)[0]
    
    # Save to a text file
    with open(f"cluster_{cluster_id}_subject_{subject_description[:20]}.txt", "w") as file:
        file.write("\n".join(cluster_sentences))

NameError: name 'sess' is not defined

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

# Load pre-trained GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Load pre-trained SentenceTransformer model
bert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Sample page of notes (replace this with your actual text)

# Tokenize the text into sentences using spaCy
sentences = [sent.text for sent in nlp(notes).sents]

# Generate GPT-2 embeddings for each sentence
def get_sentence_embeddings(sentences):
    return bert_model.encode(sentences)

# Hierarchical clustering
sentence_embeddings = get_sentence_embeddings(sentences)
cluster_model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage="ward").fit(sentence_embeddings)
num_clusters = len(np.unique(cluster_model.labels_))

# Create text files for each cluster with improved subject descriptions
for cluster_id in range(num_clusters):
    cluster_sentences = [sentences[i] for i in range(len(cluster_model.labels_)) if cluster_model.labels_[i] == cluster_id]
    
    # Generate a brief subject description using GPT-2
    subject_description = gpt2_tokenizer.decode(gpt2_model.generate(gpt2_tokenizer.encode(cluster_sentences[0], return_tensors="pt"), max_length=50)[0], skip_special_tokens=True)
    
    # Save to a text file
    with open(f"cluster_{cluster_id}_subject_{subject_description[:20]}.txt", "w") as file:
        file.write("\n".join(cluster_sentences))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 61, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.